# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [69]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Output File (CSV)
output_data_file = "output_data/Hotels.csv"

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [70]:
# Save path to SFH_data set in a variable
city_data_file = "../WeatherPY/output_data/cities.csv"
city_data_df = pd.read_csv(city_data_file)
city_data_df.dtypes

Unnamed: 0      int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [71]:
# generate some (latitude, longitude) pairs
locations = city_data_df[["Lat", "Lng"]]
center_map = ["0", "0"]

In [72]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure(map_type='ROADMAP')

heatmap_layer = gmaps.heatmap_layer(
    city_data_df[["Lat", "Lng"]], weights=city_data_df["Humidity"],
    max_intensity=100, point_radius=7.5)
fig = gmaps.figure(center=center_map, zoom_level=1.9, layout = {'width' : '100%', 'height' : '675px'})
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='675px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [73]:
#Narrow down the DataFrame to find your ideal weather condition. 
# A max temperature lower than 75 degrees but higher than 55.
#Humidity < 40%
#Wind speed less than 7 mph.
prime_weather_df = city_data_df[city_data_df["Max Temp"] <75]
prime_weather_df = prime_weather_df[prime_weather_df["Max Temp"]>55]
prime_weather_df = prime_weather_df[prime_weather_df["Humidity"]<40]
prime_weather_df = prime_weather_df[prime_weather_df["Wind Speed"]<7]
prime_weather_df.dropna()
prime_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,Zakamensk,50.37,103.29,72.72,22,12,1.90,RU,1587953091
80,80,Khovd,48.01,91.64,65.52,12,5,2.59,MN,1587952848
161,161,Ulaangom,49.98,92.07,64.62,19,97,3.98,MN,1587953148
168,168,Āsosa,10.07,34.53,69.62,31,90,2.86,ET,1587953151
181,181,Kutum,14.20,24.67,69.49,18,47,5.59,SD,1587953156
198,198,Xining,36.62,101.77,57.20,12,0,1.52,CN,1587953163
229,229,Tumut,-35.30,148.22,65.34,37,1,3.78,AU,1587953177
256,256,Pingdu,36.78,119.95,62.60,29,0,4.47,CN,1587953188
257,257,Orlik,52.52,99.83,57.63,23,100,1.79,RU,1587953189
284,284,Huanren,41.26,125.37,55.38,34,43,4.43,CN,1587953200


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [74]:
hotel_df = prime_weather_df
hotel_df["Hotel Name"] = ""
hotel_df.reset_index(inplace=True)
hotel_df= hotel_df.drop(columns='index')
lat = hotel_df["Lat"].astype(float)
lng = hotel_df["Lng"].astype(float)
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,20,Zakamensk,50.37,103.29,72.72,22,12,1.90,RU,1587953091,
1,80,Khovd,48.01,91.64,65.52,12,5,2.59,MN,1587952848,
2,161,Ulaangom,49.98,92.07,64.62,19,97,3.98,MN,1587953148,
3,168,Āsosa,10.07,34.53,69.62,31,90,2.86,ET,1587953151,
4,181,Kutum,14.20,24.67,69.49,18,47,5.59,SD,1587953156,


In [75]:
lat = hotel_df["Lat"]
lng = hotel_df["Lng"]
hotels=[]    

for i in range (len(hotel_df)):
    target_type = "lodging"
    location = f"{lat[i]},{lng[i]}"
    keyword= "hotel"
    radius = 50000
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params= {
    "location" : location,
    "radius" : radius,
    "types": target_type,
    "key" : g_key
    }
            
    try:
        response = requests.get(base_url, params=params).json()
        #pprint(response)
        hotel_name = response
        hotels.append(hotel_name["results"][0]["name"])
        print(hotel_name["results"][0]["name"])
    except:
        hotels.append("")
        print("No result found. Skipping ... ")
        pass

Khostel Zakamensk
Hotel Khovd
Achit Nuur Hotel & Restaurant
Blendana Hotel Assosa
Dibbo areaمنطقة دبو
Super Leisure Hotel
Comfort Inn Sovereign Gundagai
Duowei Boutique Hotel Pingdu
Oka
Dynasty Wunvshan Ice Chateau
No result found. Skipping ... 
Avenue GuestHouse
Hilton Garden Inn Tri-Cities/Kennewick
Baza Otdykha Sibirskaya Skazka
Albustan Luxurious Suites
JJ Inns - Yinchuan New Moon Square Branch
No result found. Skipping ... 
塔河宾馆
Mayak
Water Holiday Resort
Beach Albatros Resort


In [76]:
#drop na 
hotel_df.dropna()
hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,20,Zakamensk,50.37,103.29,72.72,22,12,1.90,RU,1587953091,Khostel Zakamensk
1,80,Khovd,48.01,91.64,65.52,12,5,2.59,MN,1587952848,Hotel Khovd
2,161,Ulaangom,49.98,92.07,64.62,19,97,3.98,MN,1587953148,Achit Nuur Hotel & Restaurant
3,168,Āsosa,10.07,34.53,69.62,31,90,2.86,ET,1587953151,Blendana Hotel Assosa
4,181,Kutum,14.20,24.67,69.49,18,47,5.59,SD,1587953156,Dibbo areaمنطقة دبو
5,198,Xining,36.62,101.77,57.20,12,0,1.52,CN,1587953163,Super Leisure Hotel
6,229,Tumut,-35.30,148.22,65.34,37,1,3.78,AU,1587953177,Comfort Inn Sovereign Gundagai
7,256,Pingdu,36.78,119.95,62.60,29,0,4.47,CN,1587953188,Duowei Boutique Hotel Pingdu
8,257,Orlik,52.52,99.83,57.63,23,100,1.79,RU,1587953189,Oka
9,284,Huanren,41.26,125.37,55.38,34,43,4.43,CN,1587953200,Dynasty Wunvshan Ice Chateau


In [77]:
#Save Hotel_df to csv
hotel_df.to_csv(output_data_file)

#Create Variables for marker info
City = hotel_df["City"]
HotelName = hotel_df["Hotel Name"]


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [79]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='675px', width='100%'))